In [11]:
import pandas as pd
import numpy as np
from pyfaidx import Fasta
import h5py

In [12]:
BEDCOLS = [
    "chrom",
    "chromStart",
    "chromEnd",
    "name",
    "score",
    "strand",
    "thickStart",
    "thickEnd",
    "itemRGB",
    "blockCount",
    "blockSizes",
    "blockStarts",
]


def generate_colnames(df, labelnum=0):  # need to be adjusted for GC content
    """Generates column names for an input dataframe.

    Arguments:
        df [dataframe] -- [dataframe for which column names are generated]

    Returns:
        [list] -- [list of generated column names]
    """
    colnames = []
    for field in range(len(df.columns) - labelnum):
        colnames.append(BEDCOLS[field])
    for label in range(labelnum):
        colnames.append(f"label_{label+1}")
    return colnames


def read_bed_file(path, labelnum=0):
    """reads .bed file into pandas dataframe

    Args:
        path (str): path to bed file

    Returns:
        [pandas dataframe]: dataframe containing all fields contained in bed file
    """

    bed_df = pd.read_table(path, sep="\t", header=None)
    colnames = generate_colnames(bed_df, labelnum)
    bed_df.columns = colnames
    print(bed_df.head())
    return bed_df


def one_hot_encoding(sequence):
    """One hot encoding of a DNA sequence.

    Args:
        sequence (str): [DNA sequence in IUPAC format]

    Returns:
        [numpy array]: [one hot encoded DNA sequence]
    """

    mydict = {
        "A": np.asarray([1, 0, 0, 0]),
        "a": np.asarray([1, 0, 0, 0]),
        "C": np.asarray([0, 1, 0, 0]),
        "c": np.asarray([0, 1, 0, 0]),
        "G": np.asarray([0, 0, 1, 0]),
        "g": np.asarray([0, 0, 1, 0]),
        "T": np.asarray([0, 0, 0, 1]),
        "t": np.asarray([0, 0, 0, 1]),
        "Y": np.asarray([0, 1, 0, 1]),
        "y": np.asarray([0, 1, 0, 1]),
        "R": np.asarray([1, 0, 1, 0]),
        "r": np.asarray([1, 0, 1, 0]),
        "S": np.asarray([0, 1, 1, 0]),
        "s": np.asarray([0, 1, 1, 0]),
        "W": np.asarray([1, 0, 0, 1]),
        "w": np.asarray([1, 0, 0, 1]),
        "K": np.asarray([0, 0, 1, 1]),
        "k": np.asarray([0, 0, 1, 1]),
        "M": np.asarray([1, 1, 0, 0]),
        "m": np.asarray([1, 1, 0, 0]),
        "B": np.asarray([0, 1, 1, 1]),
        "b": np.asarray([0, 1, 1, 1]),
        "D": np.asarray([1, 0, 1, 1]),
        "d": np.asarray([1, 0, 1, 1]),
        "H": np.asarray([1, 1, 0, 1]),
        "h": np.asarray([1, 1, 0, 1]),
        "V": np.asarray([1, 1, 1, 0]),
        "v": np.asarray([1, 1, 1, 0]),
        "N": np.asarray([0, 0, 0, 0]),
        "n": np.asarray([0, 0, 0, 0]),
        "-": np.asarray([0, 0, 0, 0]),
    }
    print(f"Seq: {sequence}")
    if len(sequence) > 0:
        nuc_list = list()
        for nuc in list(sequence):
            nuc_list.append(mydict[nuc])
        result = np.stack(np.asarray(nuc_list, dtype="int8"))
        return result
    else: 
        print("ERROR! sequence is too short")

In [15]:
def bed_encoding(bed_df, reference):
    """One Hot encoding of regions in a bed file.

    Args:
        bed_df (pandas Dataframe): [Dataframe of bed file]
        reference (fasta file): [Genome reference file in fasta format]

    Returns:
        [numpy array]: [One hot encoded DNA sequences from bed file]
    """

    fasta = Fasta(reference, as_raw=True)
    seq_list = list()
    for _, i in bed_df.iterrows():
        print(f"region:{i[0]}:{i[1]}-{i[2]}")
        seq_list.append(one_hot_encoding(fasta[i[0]][i[1]:i[2]]))
    result = np.stack(seq_list)
    return result

In [71]:
def train_validate_test_split(bed_df, v_holdout, t_holdout, ref):
    x_train = y_train = x_val = y_val = x_test = y_test = None
    
    if t_holdout is not None and v_holdout is not None:
        train_df = bed_df.loc[~bed_df["chrom"].isin(t_holdout + v_holdout)]
        validate_df = bed_df.loc[bed_df["chrom"].isin(v_holdout)]
        test_df = bed_df.loc[bed_df["chrom"].isin(t_holdout)]
        x_train = bed_encoding(train_df, ref)
        y_train = np.asarray(
            train_df.loc[:, train_df.columns.str.contains("label")], dtype="int8"
        )  # .flatten()
        x_val = bed_encoding(validate_df, ref)
        y_val = np.asarray(
            validate_df.loc[:, validate_df.columns.str.contains("label")], dtype="int8"
        )  # .flatten()
        x_test = bed_encoding(test_df, ref)
        y_test = np.asarray(
            test_df.loc[:, test_df.columns.str.contains("label")], dtype="int8"
        ) 
        
    elif t_holdout is not None:
        train_df = bed_df.loc[~bed_df["chrom"].isin(t_holdout)]
        test_df = bed_df.loc[bed_df["chrom"].isin(t_holdout)]
        x_train = bed_encoding(train_df, ref)
        y_train = np.asarray(
            train_df.loc[:, train_df.columns.str.contains("label")], dtype="int8"
        )  # .flatten()
        x_test = bed_encoding(test_df, ref)
        y_test = np.asarray(
            test_df.loc[:, test_df.columns.str.contains("label")], dtype="int8"
        )  # .flatten()
    elif v_holdout is not None:
        train_df = bed_df.loc[~bed_df["chrom"].isin(v_holdout)]
        validate_df = bed_df.loc[bed_df["chrom"].isin(v_holdout)]
        x_train = bed_encoding(train_df, ref)
        y_train = np.asarray(
            train_df.loc[:, train_df.columns.str.contains("label")], dtype="int8"
        )  # .flatten()
        x_val = bed_encoding(validate_df, ref)
        y_val = np.asarray(
            validate_df.loc[:, validate_df.columns.str.contains("label")], dtype="int8"
        )  # .flatten()
    else:
        train_df = bed_df
        x_train = bed_encoding(train_df, ref)
        y_train = np.asarray(
            train_df.loc[:, train_df.columns.str.contains("label")], dtype="int8"
        )  # .flatten()
    
    
    return x_train, y_train, x_val, y_val, x_test, y_test

In [40]:
def save_to_hd5(out_file, x_train, y_train, x_val, y_val, x_test, y_test):
    """Writes train,validate,test arrays to HD5 format"""
    data = h5py.File(out_file, "w")
    train_data = data.create_group("train_data")
    train_data.create_dataset("x_train", data=x_train)
    train_data.create_dataset("y_train", data=y_train)
    if x_val is not None:
        val_data = data.create_group("val_data")
        val_data.create_dataset("x_val", data=x_val)
        val_data.create_dataset("y_val", data=y_val)
    if x_test is not None:
        test_data = data.create_group("test_data")
        test_data.create_dataset("x_test", data=x_test)
        test_data.create_dataset("y_test", data=y_test)
    data.close()

In [39]:
def bed_to_1hot(input_file, output_file, reference, label_num, v_holdout, t_holdout):
    """Takes a bedfile, queries a fasta file and saves the one hot encoded sequences to hd5."""
    bed_df = read_bed_file(input_file, label_num)
    # print("generating data split")
    print(bed_df.head())
    x_train, y_train, x_val, y_val, x_test, y_test = train_validate_test_split(
        bed_df=bed_df, ref=reference, v_holdout=v_holdout, t_holdout=t_holdout
    )
    save_to_hd5(
        out_file=output_file,
        x_train=x_train,
        y_train=y_train,
        x_val=x_val,
        y_val=y_val,
        x_test=x_test,
        y_test=y_test,
    )

In [41]:
input_file="tests/test_data/encoding_test_2label.bed"
output_file="test.h5"
reference="/home/user/git_private/data/reference/hs38.fa"
label_num=2

In [52]:
t_holdout=None
v_holdout=None

In [44]:
t_holdout=["chr1"]
v_holdout=["chr2"]

In [69]:
t_holdout=None
v_holdout=["chr2"]

In [80]:
t_holdout=["chr1"]
v_holdout=None

In [82]:
bed_to_1hot(input_file=input_file, output_file=output_file, reference=reference, label_num=label_num, v_holdout=v_holdout, t_holdout=t_holdout)

   chrom  chromStart   chromEnd  name  score strand  label_1  label_2
0  chr18    17431849   17431859     1     10      -        0        1
1   chr5    21210294   21210304     2     10      +        1        0
2   chr6    94689432   94689442     3     10      +        0        1
3   chr2   215518670  215518680     4     10      -        1        0
4   chr9    93351305   93351315     5     10      +        0        1
   chrom  chromStart   chromEnd  name  score strand  label_1  label_2
0  chr18    17431849   17431859     1     10      -        0        1
1   chr5    21210294   21210304     2     10      +        1        0
2   chr6    94689432   94689442     3     10      +        0        1
3   chr2   215518670  215518680     4     10      -        1        0
4   chr9    93351305   93351315     5     10      +        0        1
region:chr18:17431849-17431859
Seq: AGAGTTGAAC
region:chr5:21210294-21210304
Seq: ATTGTGCCAT
region:chr6:94689432-94689442
Seq: CCAAGCAGAA
region:chr2:215518670

In [83]:
dt=h5py.File("test.h5","r")

In [84]:
dt.keys()

<KeysViewHDF5 ['test_data', 'train_data']>

In [85]:
dt["train_data"].keys()

<KeysViewHDF5 ['x_train', 'y_train']>

In [86]:
dt["train_data"]["x_train"]

<HDF5 dataset "x_train": shape (9, 10, 4), type "|i1">

In [87]:
dt["val_data"].keys()

KeyError: "Unable to open object (object 'val_data' doesn't exist)"

In [88]:
dt["val_data"]["x_val"]

KeyError: "Unable to open object (object 'val_data' doesn't exist)"

In [89]:
dt["test_data"].keys()

<KeysViewHDF5 ['x_test', 'y_test']>

In [90]:
dt["test_data"]["x_test"]

<HDF5 dataset "x_test": shape (1, 10, 4), type "|i1">